# Analysis of Tweets from a full archival search

In [1]:
import pandas as pd
from os.path import join
import numpy as np

In [1]:
#!pip install --upgrade twarc

In [2]:
# Define the main folder and files (hashtags are the files).
src = (r'C:\Users\Laser\Desktop\code_manuals') # put the path here. In windows, use: r'\...'
hashtags = ['Shell'] # Put the file name here

Note: if the Tweets have already been downloaded and the Tweet data exists in the folder ```data``` as compressed ```.jsonl``` files, you can skip the "Query tweets" and "Compress data" steps and start processing at "Decompress data".

## Collect Tweets

### Query tweets

In [3]:
# Change file permissions such that execution is allowed
! chmod +x r'C:\Users\Laser\Desktop\code_manuals\queries\shell.sh'

# Run the query. Note: this can take a while, depending on the number of Tweets
# that need to be downloaded
! r'C:\Users\Laser\Desktop\code_manuals\queries\shell.sh'

Der Befehl "chmod" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.
Die Syntax f�r den Dateinamen, Verzeichnisnamen oder die Datentr„gerbezeichnung ist falsch.


In [13]:
! chmod +x /media/s/Linux_storage/Analyse_Verkehrswende_Transformation/Code/Queries/Digitale_grüne_Bahn.sh
! /media/s/Linux_storage/Analyse_Verkehrswende_Transformation/Code/Queries/Digitale_grüne_Bahn.sh

In [ ]:
C:\Users\Laser\Desktop\code_manuals\queries>twarc2 search --start-time 2018-01-01 --end-time 2021-07-06 --archive "greenwashing (to:shell)" --max-results 100 > C:\Users\Laser\Desktop\code_manuals\queries\shell.jsonl

C:\Users\Laser\Desktop\code_manuals\queries>twarc2 search --start-time 2011-06-01 --end-time 2021-07-06 --archive "greenwashing (to:db_bahn)" --max-results 100 > C:\Users\Laser\Desktop\code_manuals\queries\db.jsonl

C:\Users\Laser\Desktop\code_manuals\queries>twarc2 search --start-time 2015-06-01 --end-time 2021-07-06 --archive "greenwashing (to:volkswagen)" --max-results 100 > C:\Users\Laser\Desktop\code_manuals\queries\vw.jsonl

In [4]:
twarc2 counts "greenwashing OR Greenwashing OR GREENWASHING" greenwashing.csv --archive --csv --granularity day

SyntaxError: invalid syntax (<ipython-input-4-21d9b4bcf8e0>, line 1)

### Compress data

Note: under windows, .xz files can be decompressed for examply with [WinZIP](https://www.winzip.com/win/en/xz-file.html).

In [27]:
# the parameter "-k" keeps the original file
! xz -k /media/s/Linux_storage/Analyse_Verkehrswende_Transformation/Data/Verkehrswende.jsonl

### Decompress data

In [ ]:
! xz -d /media/s/Linux_storage/Analyse_Verkehrswende_Transformation/Data/Verkehrswende.jsonl.xz

### Convert to CSV

Removes duplicate tweets (by ID) but keeps referenced tweets.

In [6]:
! twarc2 csv --extra-input-columns "in_reply_to_user.withheld.scope" "C:\Users\Laser\Desktop\code_manuals\queries\shell.jsonl" "C:\Users\Laser\Desktop\code_manuals\queries\shell.csv"


  0%|          | 0.00/1.36M [00:00<?, ?B/s]
100%|##########| 1.36M/1.36M [00:00<00:00, 2.58MB/s]

\u2139\ufe0f
Read 822 tweets from 5 lines. 
399 were referenced tweets, 203 were duplicates.
Wrote 619 rows and output 90 of 90 input columns in the CSV.



In [7]:
! twarc2 csv --extra-input-columns "in_reply_to_user.withheld.scope" "C:\Users\Laser\Desktop\code_manuals\queries\hm.jsonl" "C:\Users\Laser\Desktop\code_manuals\queries\hm.csv


  0%|          | 0.00/280k [00:00<?, ?B/s]
100%|##########| 280k/280k [00:00<00:00, 1.86MB/s]

\u2139\ufe0f
Read 115 tweets from 1 lines. 
46 were referenced tweets, 20 were duplicates.
Wrote 95 rows and output 90 of 90 input columns in the CSV.



In [8]:
! twarc2 csv --extra-input-columns "in_reply_to_user.withheld.scope" "C:\Users\Laser\Desktop\code_manuals\queries\db.jsonl" "C:\Users\Laser\Desktop\code_manuals\queries\db.csv


  0%|          | 0.00/25.8k [00:00<?, ?B/s]
100%|##########| 25.8k/25.8k [00:00<00:00, 506kB/s]

\u2139\ufe0f
Read 19 tweets from 1 lines. 
9 were referenced tweets, 2 were duplicates.
Wrote 17 rows and output 90 of 90 input columns in the CSV.



In [9]:
! twarc2 csv --extra-input-columns "in_reply_to_user.withheld.scope" "C:\Users\Laser\Desktop\code_manuals\queries\bahn.jsonl" "C:\Users\Laser\Desktop\code_manuals\queries\bahn.csv


  0%|          | 0.00/1.53M [00:00<?, ?B/s]
100%|##########| 1.53M/1.53M [00:00<00:00, 2.75MB/s]

\u2139\ufe0f
Read 1327 tweets from 8 lines. 
609 were referenced tweets, 492 were duplicates.
Wrote 835 rows and output 90 of 90 input columns in the CSV.



## Extract conversation IDs

Be aware that all necessary folders need to be existsting (case sensitive). Reduce chunksizes to get smaller .txt files. These can be quite long lists, maybe it makes more sense to work with the most active conversations. At least it is helpful to get the amount of tweets relative to conversations (the higher the tweets, the more intense the conversations are).

In [31]:
def get_conversation_IDs(src, filename, chunks=False, chunksize=10000):
    data = pd.read_csv(join(src, '{}.csv'.format(filename)), low_memory=False)
    conversationIDs = data['conversation_id'].dropna().astype(int).unique()
    print('{}: There are {} Tweets from {} conversations'\
              .format(filename, len(data), len(conversationIDs)))
    
    dst = join(src, 'conversation_IDs')
    
    if chunks:
        N_chunks = len(conversationIDs) // chunksize
        print(N_chunks)
        for i in range(N_chunks):
            ID_chunk = conversationIDs[i * chunksize : (i + 1) * chunksize]
            np.savetxt(join(dst, '{}_ConversationIDs_{}_to_{}.txt'\
                .format(filename, i * chunksize, (i + 1) * chunksize)),
                ID_chunk, fmt='%d')
        np.savetxt(join(dst, '{}_ConversationIDs_{}_to_{}.txt'\
                .format(filename, N_chunks * chunksize, len(conversationIDs))),
                conversationIDs[N_chunks * chunksize : ], fmt='%d')
            
    else:   
        np.savetxt(join(dst, '{}_ConversationIDs.txt'.format(filename)),
                   conversationIDs, fmt='%d')

In [32]:
get_conversation_IDs(src, 'Verkehrswende', chunks=True)

Verkehrswende: There are 47489 Tweets from 38917 conversations
38


## Extract Tweet IDs

In [14]:
def get_Tweet_IDs(src, filename, chunks=False, chunksize=1000):
    data = pd.read_csv(join(src, '{}.csv'.format(filename)), low_memory=False)
    TweetIDs = data['id'].dropna().astype(int).unique()
    print('{}: There are {} Tweets'\
              .format(filename, len(TweetIDs)))
    
    dst = join(src, 'tweet_IDs')
    
    if chunks:
        N_chunks = len(TweetIDs) // chunksize
        print(N_chunks)
        for i in range(N_chunks):
            ID_chunk = TweetIDs[i * chunksize : (i + 1) * chunksize]
            np.savetxt(join(dst, '{}_TweetIDs_{}_to_{}.txt'\
                .format(filename, i * chunksize, (i + 1) * chunksize)),
                ID_chunk, fmt='%d')
        np.savetxt(join(dst, '{}_TweetIDs_{}_to_{}.txt'\
                .format(filename, N_chunks * chunksize, len(TweetIDs))),
                TweetIDs[N_chunks * chunksize : ], fmt='%d')
            
    else:   
        np.savetxt(join(dst, '{}_TweetIDs.txt'.format(filename)),
                   TweetIDs, fmt='%d')

In [15]:
get_Tweet_IDs(src, 'IchBinHanna', chunks=False)

IchBinHanna: There are 75910 Tweets
